In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import gradio as gr
import time
import gtts
from playsound import playsound
from pygame import mixer  # Load the popular external library


c:\Users\amvk2\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()

In [3]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,max_new_tokens=2000)  


In [5]:
print(tokenizer.decode(outputs[0]))

<pad> Wie old sind Sie?</s>


In [6]:
mixer.music.load('hello.mp3')
mixer.music.play()

error: No file 'hello.mp3' found in working directory 'c:\Users\amvk2\OneDrive\Documents\30-days-python-challenge\day15'.

In [9]:
count = 1

In [10]:
def func_prompt(message):
    input_text = f"""Q: {message}
    A: """
    input_ids = tokenizer(input_text, return_tensors="pt",).input_ids
    outputs = model.generate(input_ids, max_new_tokens=4000)
    answer = (tokenizer.decode(outputs[0])).replace("<pad>", "").replace("</s>", "")
    global count
    count += 1
    tts = gtts.gTTS(answer)
    tts.save(f"audio/voice{count}.mp3")
    mixer.music.load(f'audio/voice{count}.mp3')
    mixer.music.play()
    
    return answer

In [11]:
func_prompt("what is a gun")

' A gun is a firearm that is used to shoot or fire a weapon.'

In [13]:
with gr.Blocks() as demo:
    desc = gr.Markdown(""" 
    # Chat Bot with voice using Google/flan-t5-base
                       
    made by arunmozhi varman
    """)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = func_prompt(message)
        chat_history.append((message, bot_message))
        time.sleep(2)
        
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.27.0, however version 4.29.0 is available, please upgrade.
--------
